In [6]:
# FP树类
import time


class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue  #节点元素名称，在构造时初始化为给定值
        self.count = numOccur   # 出现次数，在构造时初始化为给定值
        self.nodeLink = None   # 指向下一个相似节点的指针，默认为None
        self.parent = parentNode   # 指向父节点的指针，在构造时初始化为给定值
        self.children = {}  # 指向子节点的字典，以子节点的元素名称为键，指向子节点的指针为值，初始化为空字典

    # 增加节点的出现次数值
    def inc(self, numOccur):
        self.count += numOccur

    # 输出节点和子节点的FP树结构
    def disp(self, ind=1):
        print(' ' * ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind + 1)
            
    def count(self,keys,ind=1,sum=0):
        if self.name==keys:
            sum=self.count+sum
            print(sum)
        for child in self.children.values():
            child.count(keys,ind + 1,sum)
            print(sum)
        

# =======================================================构建FP树==================================================

# 对不是第一个出现的节点，更新头指针块。就是添加到相似元素链表的尾部
def updateHeader(nodeToTest, targetNode):
    while (nodeToTest.nodeLink != None):
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

# 根据一个排序过滤后的频繁项更新FP树
def updateTree(items, inTree, headerTable, count):
    if items[0] in inTree.children:
        # 有该元素项时计数值+1
        inTree.children[items[0]].inc(count)
    else:
        # 没有这个元素项时创建一个新节点
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        # 更新头指针表或前一个相似元素项节点的指针指向新节点
        if headerTable[items[0]][1] == None:  # 如果是第一次出现，则在头指针表中增加对该节点的指向
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])

    if len(items) > 1:
        # 对剩下的元素项迭代调用updateTree函数
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)

# 主程序。创建FP树。dataSet为事务集，为一个字典，键为每个事物，值为该事物出现的次数。minSup为最低支持度
def createTree(dataSet, minSup=1):
    # 第一次遍历数据集，创建头指针表
    headerTable = {}
    for trans in dataSet:
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    # 移除不满足最小支持度的元素项
    keys = list(headerTable.keys()) # 因为字典要求在迭代中不能修改，所以转化为列表
    for k in keys:
        if headerTable[k] < minSup:
            del(headerTable[k])
    # 空元素集，返回空
    freqItemSet = set(headerTable.keys())
    if len(freqItemSet) == 0:
        return None, None
    # 增加一个数据项，用于存放指向相似元素项指针
    for k in headerTable:
        headerTable[k] = [headerTable[k], None]  # 每个键的值，第一个为个数，第二个为下一个节点的位置
    retTree = treeNode('Null Set', 1, None) # 根节点
    # 第二次遍历数据集，创建FP树
    for tranSet, count in dataSet.items():
        localD = {} # 记录频繁1项集的全局频率，用于排序
        for item in tranSet:
            if item in freqItemSet:   # 只考虑频繁项
                localD[item] = headerTable[item][0] # 注意这个[0]，因为之前加过一个数据项
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)] # 排序
            updateTree(orderedItems, retTree, headerTable, count) # 更新FP树
    return retTree, headerTable

# =================================================查找元素条件模式基===============================================

# 直接修改prefixPath的值，将当前节点leafNode添加到prefixPath的末尾，然后递归添加其父节点。
# prefixPath就是一条从treeNode（包括treeNode）到根节点（不包括根节点）的路径
def ascendTree(leafNode, prefixPath):
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)

# 为给定元素项生成一个条件模式基（前缀路径）。basePet表示输入的频繁项，treeNode为当前FP树中对应的第一个节点
# 函数返回值即为条件模式基condPats，用一个字典表示，键为前缀路径，值为计数值。
def findPrefixPath(basePat, treeNode):
    condPats = {}  # 存储条件模式基
    while treeNode != None:
        prefixPath = []  # 用于存储前缀路径
        ascendTree(treeNode, prefixPath)  # 生成前缀路径
        if len(prefixPath) > 1:
            condPats[frozenset(prefixPath[1:])] = treeNode.count  # 出现的数量就是当前叶子节点的数量
        treeNode = treeNode.nodeLink  # 遍历下一个相同元素
    return condPats

# =================================================递归查找频繁项集===============================================
# 根据事务集获取FP树和频繁项。
# 遍历频繁项，生成每个频繁项的条件FP树和条件FP树的频繁项
# 这样每个频繁项与他条件FP树的频繁项都构成了频繁项集

# inTree和headerTable是由createTree()函数生成的事务集的FP树。
# minSup表示最小支持度。
# preFix请传入一个空集合（set([])），将在函数中用于保存当前前缀。
# freqItemList请传入一个空列表（[]），将用来储存生成的频繁项集。
def mineTree(inTree, headerTable, minSup, preFix, freqItemList):
    # 对频繁项按出现的数量进行排序进行排序
    sorted_headerTable = sorted(headerTable.items(), key=lambda p: p[1][0])  #返回重新排序的列表。每个元素是一个元组，[（key,[num,treeNode],()）
    bigL = [v[0] for v in sorted_headerTable]  # 获取频繁项
    
    for basePat in bigL:
        newFreqSet = preFix.copy()  # 新的频繁项集
        newFreqSet.add(basePat)     # 当前前缀添加一个新元素
        freqItemList.append(newFreqSet)  # 所有的频繁项集列表
        condPattBases = findPrefixPath(basePat, headerTable[basePat][1])  # 获取条件模式基。就是basePat元素的所有前缀路径。它像一个新的事务集
        myCondTree, myHead = createTree(condPattBases, minSup)  # 创建条件FP树

        if myHead != None:
            # 用于测试
            #print('conditional tree for:', newFreqSet)
            #myCondTree.disp()
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)  # 递归直到不再有元素


# 将数据集转化为目标格式
def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    #print("******************retDict*********************")
    #print(retDict)
    return retDict

import pandas as pa
import numpy as np

#读取IBM中生成数据
def loadData():
    Data = pa.read_csv("FBtree_TestingData.csv")       #使用pandas读取数据

    newData=np.zeros((len(Data,),2),dtype=int)
    for i in range(len(Data)):                       
        newData[i]=Data['items'][i].split(" ")           #对数据进行切割

    newdata=[]
    temp=[]
    n=1
    for j in range(len(Data)):                         #按照交易次数重组数据
        if newData[j][0]==n:
            temp.insert(0,newData[j][1])
        else:
            newdata.insert(0,temp)
            temp=[]
            temp.insert(0,newData[j][1])
            n=n+1
    print("********************newdata*******************")
    print(newdata)
    return newdata

if __name__=='__main__':
    
    start = time.clock()
    minSup =8
    simpDat = loadData()  # 加载数据集
    initSet = createInitSet(simpDat)  # 转化为符合格式的事务集
    myFPtree, myHeaderTab = createTree(initSet, minSup)  # 形成FP树
    print("******************myFPtree*********************")
    #myFPtree.disp()  # 打印树
    
    print("------------------myFPtree---------------------")

    freqItems = []  # 用于存储频繁项集
    mineTree(myFPtree, myHeaderTab, minSup, set([]), freqItems)  # 获取频繁项集
    print("******************繁项集*********************")
    number=0
    for i in range(len(freqItems)):
        if len(freqItems[i])!=1:
            print(freqItems[i])
            number=number+1
    print("总共有频集：",number,"个")
    end = time.clock()
    print("程序运行时间为：",end-start)
 


********************newdata*******************
[[954, 826, 703, 592, 154], [963, 545, 230, 125, 5], [860, 781, 629, 238], [883, 738, 444, 375, 272, 197, 67], [942, 907, 402, 336, 74], [876, 699, 592, 267, 87, 51], [990, 132], [744, 518, 336, 263, 35, 28, 3], [962, 782, 622, 423], [357, 59], [944, 613, 595], [592, 540, 319, 76], [881, 532, 20], [947, 868, 543, 495, 319, 311, 132], [973, 938, 773, 665, 481, 146], [966, 941, 868, 709, 656, 646, 591, 388, 123, 81], [391, 123, 36, 26], [916, 907, 789, 743, 314, 261], [592, 374], [981, 753, 682, 571, 543, 540, 412, 7], [976, 808, 800, 717, 414, 127, 102], [547, 26], [831, 800, 729, 708, 495, 222, 200], [948, 874, 521, 86, 62], [992, 776, 747, 734, 682, 646, 544, 402, 316, 53], [837, 619, 612, 464, 259, 39, 31], [902, 168], [769, 513, 455, 171], [884, 653, 495, 482, 403, 170, 160, 146, 132], [952, 903, 718, 669, 547, 147], [914, 837, 795, 490, 238, 29], [917, 859, 825, 399, 214, 109, 66, 47], [678, 674, 444, 87, 62], [599, 571, 300, 53], [871

{682, 820}
{753, 682}
{874, 699}
{925, 807}
{994, 963}
{456, 371}
{800, 402}
{405, 374}
{442, 871}
{377, 553}
{545, 471}
{416, 124}
{371, 444}
{374, 238}
{442, 707}
{764, 709}
{360, 592}
{360, 63}
{553, 988}
{800, 222}
{709, 221}
{444, 238}
{545, 38}
{592, 827}
{416, 545}
{432, 592}
{800, 601}
总共有频集： 58 个
程序运行时间为： 0.549964205489232
